In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
##DATA INGESTION-from the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader(web_path=("https://python.langchain.com/docs/integrations/providers/google/"),)
docs=loader.load()
docs



/Users/ayushsiddhant/Desktop/Langchain/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://python.langchain.com/docs/integrations/providers/google/', 'title': 'Google | 🦜️🔗 LangChain', 'description': 'All functionality related to Google Cloud, Google Gemini and other Google products.', 'language': 'en'}, page_content='\n\n\n\n\nGoogle | 🦜️🔗 LangChain\n\n\n\n\n\n\n\n\nSkip to main content⚠️ THESE DOCS ARE OUTDATED. Visit the new v1.0 docsIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchProvidersAnthropicAWSGoogleHugging FaceMicrosoftOpenAIMoreProvidersAbsoAcreomActiveloop Deep LakeADS4GPTsAgentQLAI21 LabsAimAI/ML API LLMAINetworkAirbyteAirtableAlchemyAleph AlphaAlibaba CloudAnalyticDBAnchor BrowserAnnoyAnthropicAnyscaleApache Software FoundationApache DorisApifyAppleArangoDBArceeArcGISArgillaArizeArthurArxivAscendAskNewsAssemblyAIAstra DBAtlasAwaDBAWSAZLyricsAzure AIBAAIBagelBagelDBBaichuanBaiduBananaBasetenBeamBeautiful SoupBibTeXBiliBiliBittensorBlackboar

In [3]:
##Data Transformation-Divide the data into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
document=text_splitter.split_documents(docs)

In [4]:
document

[Document(metadata={'source': 'https://python.langchain.com/docs/integrations/providers/google/', 'title': 'Google | 🦜️🔗 LangChain', 'description': 'All functionality related to Google Cloud, Google Gemini and other Google products.', 'language': 'en'}, page_content='Google | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/integrations/providers/google/', 'title': 'Google | 🦜️🔗 LangChain', 'description': 'All functionality related to Google Cloud, Google Gemini and other Google products.', 'language': 'en'}, page_content='Skip to main content⚠️ THESE DOCS ARE OUTDATED. Visit the new v1.0 docsIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchProvidersAnthropicAWSGoogleHugging FaceMicrosoftOpenAIMoreProvidersAbsoAcreomActiveloop Deep LakeADS4GPTsAgentQLAI21 LabsAimAI/ML API LLMAINetworkAirbyteAirtableAlchemyAleph AlphaAlibaba CloudAnalyticDBAnchor BrowserAnnoyAnthropicAnysca

In [5]:
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings()


In [6]:
from langchain_community.vectorstores import FAISS
vectorstoresdb=FAISS.from_documents(document,embedding)

In [7]:
vectorstoresdb

In [8]:
query="What is integration stuff here?"
result=vectorstoresdb.similarity_search(query)
result[0].page_content

'Skip to main content⚠️ THESE DOCS ARE OUTDATED. Visit the new v1.0 docsIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchProvidersAnthropicAWSGoogleHugging FaceMicrosoftOpenAIMoreProvidersAbsoAcreomActiveloop Deep LakeADS4GPTsAgentQLAI21 LabsAimAI/ML API LLMAINetworkAirbyteAirtableAlchemyAleph AlphaAlibaba CloudAnalyticDBAnchor BrowserAnnoyAnthropicAnyscaleApache Software FoundationApache DorisApifyAppleArangoDBArceeArcGISArgillaArizeArthurArxivAscendAskNewsAssemblyAIAstra DBAtlasAwaDBAWSAZLyricsAzure AIBAAIBagelBagelDBBaichuanBaiduBananaBasetenBeamBeautiful SoupBibTeXBiliBiliBittensorBlackboardbookend.aiBoxBrave SearchBreebs (Open Knowledge)Bright DataBrowserbaseBrowserlessByteDanceCassandraCerebrasCerebriumAIChaindeskChromaClarifaiClearMLClickHouseClickUpCloudflareClovaCnosDBCogneeCogniSwitchCohereCollege ConfidentialCometConfident AIConfluenceConneryContextContextual AICouchbaseCozeCrateDBC'

In [9]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-4o-mini")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12ce601a0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x12ce60980>, root_client=<openai.OpenAI object at 0x12cf28f50>, root_async_client=<openai.AsyncOpenAI object at 0x12cf2a0d0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [20]:
#Retrieval CHAIN,Document Chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
prompt=ChatPromptTemplate.from_template(
    """
    Answer the question based on the context provided.
    <context>
    {context}
    </context>
    Question: {input}
    """
)
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain



RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n    Answer the question based on the context provided.\n    <context>\n    {context}\n    </context>\n    Question: {input}\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12ce601a0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x12ce60980>, root_client=<openai.OpenAI object at 0x12cf28f50>, root_async_client=<openai.AsyncOpenAI object at 0x12cf2a0d0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [21]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Vertex AI Model Garden and other Google Cloud services via Vertex AI and specific cloud integrations",
    "context":[Document(page_content="Vertex AI Model Garden and other Google Cloud services via Vertex AI and specific cloud integrations.Vertex AI models require the langchain-google-vertexai package. Other services might require additional packages like langchain-google-community, langchain-google-cloud-sql-pg, etc.")]
})

'The context highlights that to use Vertex AI models and other Google Cloud services through Vertex AI, you need to utilize specific packages. The essential package for Vertex AI models is the `langchain-google-vertexai` package. Additionally, other services may require other packages, such as `langchain-google-community`, `langchain-google-cloud-sql-pg`, and potentially more, depending on the specific service integration needed.'

In [22]:
retriever=vectorstoresdb.as_retriever()
from langchain_classic.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain.invoke({"input":"What is integration stuff here?"})


{'input': 'What is integration stuff here?',
 'context': [Document(id='39187995-c709-470f-a5b7-f5b637151911', metadata={'source': 'https://python.langchain.com/docs/integrations/providers/google/', 'title': 'Google | 🦜️🔗 LangChain', 'description': 'All functionality related to Google Cloud, Google Gemini and other Google products.', 'language': 'en'}, page_content='Skip to main content⚠️ THESE DOCS ARE OUTDATED. Visit the new v1.0 docsIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchProvidersAnthropicAWSGoogleHugging FaceMicrosoftOpenAIMoreProvidersAbsoAcreomActiveloop Deep LakeADS4GPTsAgentQLAI21 LabsAimAI/ML API LLMAINetworkAirbyteAirtableAlchemyAleph AlphaAlibaba CloudAnalyticDBAnchor BrowserAnnoyAnthropicAnyscaleApache Software FoundationApache DorisApifyAppleArangoDBArceeArcGISArgillaArizeArthurArxivAscendAskNewsAssemblyAIAstra DBAtlasAwaDBAWSAZLyricsAzure AIBAAIBagelBagelDBBaichuanBaiduBananaBaseten